<a href="https://colab.research.google.com/github/in-dist-adversarials/in_distribution_adversarial_examples/blob/main/demos/demo_uniform_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/in-dist-adversarials/in_distribution_adversarial_examples.git

Cloning into 'in_distribution_adversarial_examples'...
remote: Enumerating objects: 572, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 572 (delta 0), reused 3 (delta 0), pack-reused 569
Receiving objects: 100% (572/572), 108.96 MiB | 20.46 MiB/s, done.
Resolving deltas: 100% (93/93), done.
Updating files: 100% (559/559), done.


In [2]:
import sys
# import utils
import pickle
# import utils
import numpy as np
import matplotlib.pyplot as plta
import torch
import random
import torch.nn as nn

import torch.distributions as D
import numpy as np
from tqdm.notebook import tqdm
import pickle
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [3]:
ROOT='/content/in_distribution_adversarial_examples'
sys.path.append(ROOT)

In [4]:
import cma

In [5]:
class MLP(nn.Module):
    def __init__(self, data_dimensions,name,seed=None):
        if seed is not None:
            torch.manual_seed(seed)
        else:
            print("Torch seed:%s"%torch.seed())
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(data_dimensions, int(data_dimensions)),
            nn.ReLU(),
            nn.Linear(int(data_dimensions), 2)
        )
        self.name = name
        self.train_accuracy = 0
        self.test_accuracy = 0
        self.dataset = []

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x

In [6]:
class CMA_info():
    def __init__(self, model_name, num_samples):
        super(CMA_info, self).__init__()
        self.model_name = model_name
        self.num_samples = num_samples
        
        self.distances = []
        self.in_dist_advs = []
        self.advs = []
        self.starts = []
    
    def summary(self):
        print('****************** CMA Summary *******************')
        print('Trained on %s points:'%self.model_name.split('_')[-1])
        print('Adversarials: %s/%s'%(len(self.advs), self.num_samples))
        print('In-distribution: %s/%s'%(len(self.in_dist_advs), len(self.advs)))
        
        avg_dist = np.mean(self.distances)
        print('Average L2 distance: %s'%(avg_dist))
        print('*'*50)

In [7]:
x1_min = -10
x1_max = 10

x2_min = 20
x2_max = 40

test_min = -1
test_max= 1
vec_1 = np.array([x1_min]*20)
vec_2 = np.array([x1_max]*20)
total_range = np.linalg.norm(vec_2-vec_1)

In [8]:
def sample_uniform(batch_size, N_dim, r1, r2):
    random_matrix = (r1 - r2) * torch.rand([batch_size, N_dim]) + r2
    return random_matrix

In [9]:
def make_dataset(dataset_size):
    sample_1 = sample_uniform(dataset_size, N_dim, x1_min, x1_max)
    sample_2 = sample_uniform(dataset_size, N_dim, x2_min, x2_max)
    
    labels_1 = torch.zeros(len(sample_1))
    labels_2 = torch.ones(len(sample_2))
    
    X = torch.vstack([sample_1, sample_2])
    Y = torch.hstack([labels_1, labels_2])
    ids = list(range(len(X)))

    random.shuffle(ids)
    train_ids = ids[:int(0.8*len(X))]
    test_ids = ids[int(0.8*len(X)):]

    # len(X)

    X_train = X[train_ids]
    X_test = X[test_ids]

    Y_train = Y[train_ids]
    Y_test = Y[test_ids]
    return X_train, X_test, Y_train, Y_test

In [10]:
def train_model(N_dim, dataset, name, seed = None):
    X_train, X_test, Y_train, Y_test = dataset
    X_train = X_train.cuda()
    X_test = X_test.cuda()
    Y_train = Y_train.cuda()
    Y_test = Y_test.cuda()
    
    if seed is not None:
        model = MLP(N_dim, name, seed).cuda()
    else:
        model = MLP(N_dim, name).cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
    loss_fn = nn.CrossEntropyLoss()
    model.dataset = dataset
    
    for epoch in tqdm(range(150)): 
        outputs = model(X_train)
        loss = loss_fn(outputs, Y_train.long())
        loss.backward()
        optimizer.step()

        predictions = torch.argmax(model(X_test), dim=1)
        accuracy = torch.sum(predictions == Y_test)/len(Y_test)
        
        train_predictions = torch.argmax(model(X_train), dim=1)
        train_accuracy = torch.sum(train_predictions == Y_train)/len(Y_train)

        if epoch %9 ==0:
            print("Epoch:%s, Train Acc:%s"%(epoch, train_accuracy))
            print("Epoch:%s, Test Acc:%s"%(epoch, accuracy))
        
        model.test_accuracy = accuracy
        model.train_accuracy = train_accuracy
    
    return model

In [11]:
def batched_train_model(N_dim, dataset, name, seed = None):
    X_train, X_test, Y_train, Y_test = dataset
    if seed is not None:
        model = MLP(N_dim, name, seed).cuda()
    else:
        model = MLP(N_dim, name).cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
    loss_fn = nn.CrossEntropyLoss()
    model.dataset = dataset
    
    BATCH_SIZE = 64000
    
    for epoch in tqdm(range(100)):
        num_batches = int(X_train.shape[0]/(BATCH_SIZE))
        
        batch_accuracies = []
        batch_train_accuracies = []
        for batch_num in range(num_batches):
            start_pt = BATCH_SIZE * batch_num
            end_pt = start_pt + BATCH_SIZE
            train_X = X_train[start_pt:end_pt]
            train_Y = Y_train[start_pt:end_pt]
            test_X = X_test
            test_Y = Y_test
            
            train_X = train_X.cuda()
            train_Y = train_Y.cuda()
            test_X = test_X.cuda()
            test_Y = test_Y.cuda()

            outputs = model(train_X)
            loss = loss_fn(outputs, train_Y.long())
            loss.backward()
            optimizer.step()

            batch_predictions = torch.argmax(model(test_X), dim=1)
            batch_accuracy = torch.sum(batch_predictions == test_Y)/len(test_Y)
            batch_accuracies.append(batch_accuracy.item())
            
            batch_train_predictions = torch.argmax(model(train_X), dim=1)
            batch_train_accuracy = torch.sum(batch_train_predictions == train_Y)/len(train_Y)
            batch_train_accuracies.append(batch_train_accuracy.item())        
        
        accuracy = np.mean(batch_accuracies)
        train_accuracy = np.mean(batch_train_accuracies)

        
        if epoch%99 == 0:
            print("Epoch:%s, Train Acc:%s"%(epoch, train_accuracy))
            print("Epoch:%s, Test Acc:%s"%(epoch, accuracy))
        
        # if epoch == 999:
        #     print("Epoch:%s, Train Acc:%s"%(epoch, train_accuracy))
        #     print("Epoch:%s, Test Acc:%s"%(epoch, accuracy))
        
        model.test_accuracy = accuracy
        model.train_accuracy = train_accuracy
    
    return model

In [12]:
def cma_objective(x_input):
    torch_x = torch.from_numpy(x_input).unsqueeze(0).float()
    output = CURRENT_MODEL(torch_x)
    pred_prob = output[0][CATEGORY_NUM].item()
    prediction = torch.argmax(output[0]).item()
    return pred_prob, prediction

In [13]:
def cma_experiment(attacked_model, num_samples = 50):
    CATEGORY_NUM
    cma_search_output = {}
    global CURRENT_MODEL
    CURRENT_MODEL = attacked_model
    print(CURRENT_MODEL.name)
    cma_output = CMA_info(CURRENT_MODEL.name, num_samples)        
    
    for i in tqdm(range(num_samples)):
        initial_pred = 1
        while initial_pred == 1:
            start_pos = sample_uniform(1, N_dim, test_min, test_max)
            output = CURRENT_MODEL(start_pos)
            initial_pred = torch.argmax(output[0]).item()
        cma_output.starts.append(start_pos)
        start_pos = start_pos[0]
        
        es = cma.CMAEvolutionStrategy(start_pos, 0.00005)
        es.optimize(cma_objective, verb_disp = False, iterations=1500, correct_prediction = CATEGORY_NUM)
        adv_offspring_ids = np.where(np.array(es.predictions) != 0)
        
        if len(adv_offspring_ids[0]) > 0:
            random_adv_offspring_id = random.choice(list(adv_offspring_ids[0]))
            random_adv_offspring = es.prediction_settings[random_adv_offspring_id]
            cma_output.advs.append(random_adv_offspring)        

            max_val = np.max(random_adv_offspring)
            if max_val < x1_max:
                cma_output.in_dist_advs.append(random_adv_offspring)
                distance = np.linalg.norm(start_pos - random_adv_offspring)
                cma_output.distances.append(distance)
    return cma_output

In [14]:
def get_distances(all_info):
    dist_means = []
    dist_stds = []
    attack_rate_means = []
    attack_rate_stds = []

    for dset in sorted(all_info[0][1].keys()):
        all_distances = []
        all_attack_rates = []

        for i in range(1):
            all_distances.extend(all_info[i][1][dset].distances)        
            num_attacks = len(all_info[i][1][dset].in_dist_advs)
            num_total = len(all_info[i][1][dset].starts)
            all_attack_rates.extend([num_attacks/num_total])

        attack_rate_means.append(np.mean(all_attack_rates))
        attack_rate_stds.append(np.std(all_attack_rates))
        dist_mean = np.mean(all_distances)
        dist_std = np.std(all_distances)
        dist_means.append(dist_mean)
        dist_stds.append(dist_std)
    
    return attack_rate_means, attack_rate_stds, dist_means, dist_stds

In [15]:
CATEGORY_NUM = 0
CURRENT_MODEL = None

all_info = []
N_dim = 20

for iteration in range(5):
    print('*'*50)
    print('Iteration %s'%iteration)
    print('*'*50)
    
    trained_models = {}
    attack_output = {}
    for dsize in [10000]:
        print('Working with with %s'%dsize)
        dset = make_dataset(dsize)
        if dsize > 64000:
            trained_models[dsize] = batched_train_model(N_dim, dset, 'dset_size_%s'%dsize)
        else:
            trained_models[dsize] = train_model(N_dim, dset, 'dset_size_%s'%dsize)
        model_to_attack = trained_models[dsize].cpu()
        attack_output[dsize] = cma_experiment(model_to_attack, 10)
    iter_info = [trained_models, attack_output]
    all_info.append(iter_info)

**************************************************
Iteration 0
**************************************************
Working with with 10000
Torch seed:5259374522410952817


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch:0, Train Acc:tensor(0.4607, device='cuda:0')
Epoch:0, Test Acc:tensor(0.4675, device='cuda:0')
Epoch:9, Train Acc:tensor(0.8499, device='cuda:0')
Epoch:9, Test Acc:tensor(0.8555, device='cuda:0')
Epoch:18, Train Acc:tensor(0.7339, device='cuda:0')
Epoch:18, Test Acc:tensor(0.7380, device='cuda:0')
Epoch:27, Train Acc:tensor(0.6771, device='cuda:0')
Epoch:27, Test Acc:tensor(0.6795, device='cuda:0')
Epoch:36, Train Acc:tensor(0.6486, device='cuda:0')
Epoch:36, Test Acc:tensor(0.6495, device='cuda:0')
Epoch:45, Train Acc:tensor(0.6344, device='cuda:0')
Epoch:45, Test Acc:tensor(0.6360, device='cuda:0')
Epoch:54, Train Acc:tensor(0.6292, device='cuda:0')
Epoch:54, Test Acc:tensor(0.6285, device='cuda:0')
Epoch:63, Train Acc:tensor(0.6286, device='cuda:0')
Epoch:63, Test Acc:tensor(0.6280, device='cuda:0')
Epoch:72, Train Acc:tensor(0.6359, device='cuda:0')
Epoch:72, Test Acc:tensor(0.6320, device='cuda:0')
Epoch:81, Train Acc:tensor(0.6536, device='cuda:0')
Epoch:81, Test Acc:tensor

  0%|          | 0/10 [00:00<?, ?it/s]

**************************************************
Iteration 1
**************************************************
Working with with 10000
Torch seed:2746285147832025403


...

  0%|          | 0/10 [00:00<?, ?it/s]

**************************************************
Iteration 4
**************************************************
Working with with 10000
Torch seed:16228634308303044656


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch:0, Train Acc:tensor(0.5673, device='cuda:0')
Epoch:0, Test Acc:tensor(0.5753, device='cuda:0')
Epoch:9, Train Acc:tensor(0.5784, device='cuda:0')
Epoch:9, Test Acc:tensor(0.5865, device='cuda:0')
Epoch:18, Train Acc:tensor(0.6095, device='cuda:0')
Epoch:18, Test Acc:tensor(0.6195, device='cuda:0')
Epoch:27, Train Acc:tensor(0.6673, device='cuda:0')
Epoch:27, Test Acc:tensor(0.6728, device='cuda:0')
Epoch:36, Train Acc:tensor(0.7427, device='cuda:0')
Epoch:36, Test Acc:tensor(0.7480, device='cuda:0')
Epoch:45, Train Acc:tensor(0.8018, device='cuda:0')
Epoch:45, Test Acc:tensor(0.8040, device='cuda:0')
Epoch:54, Train Acc:tensor(0.8797, device='cuda:0')
Epoch:54, Test Acc:tensor(0.8878, device='cuda:0')
Epoch:63, Train Acc:tensor(0.9245, device='cuda:0')
Epoch:63, Test Acc:tensor(0.9318, device='cuda:0')
Epoch:72, Train Acc:tensor(0.9577, device='cuda:0')
Epoch:72, Test Acc:tensor(0.9613, device='cuda:0')
Epoch:81, Train Acc:tensor(0.9757, device='cuda:0')
Epoch:81, Test Acc:tensor

  0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
results = get_distances(all_info)
print('******************************************************************************************')
print('Average results for 5 repetitions of models trained for 20-dimensional 10,000 data points:')
print('******************************************************************************************')
print('Attack Rate: %s\u00B1%s'%(results[0][0], results[1][0]))
print('L2 distance to adversarial: %0.02f\u00B1%0.02f'%(results[2][0],results[3][0]))


print('Normalized by range of parameters for points belonging to this class,this is %0.02f%% change'%(results[2][0]/total_range*100))

******************************************************************************************
Average results for 5 repetitions of models trained for 20-dimensional 10,000 data points:
******************************************************************************************
Attack Rate: 1.0±0.0
L2 distance to adversarial: 2.55±1.40
Normalized by range of parameters for points belonging to this class,this is 2.85% change
